In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/stat/rushing/season/2021/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [2]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [3]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [4]:
stats_columns = ['Position', 'Games Played', 'Rushing Attempts', 'Rushing Yards',
                 'Avg Yards per Rush Attempt', 'Long Rushing', '20+ Yard Rushes', 'Rushign Touchdowns',
                 'Rushing Yards per Game', 'Rushing Fumbles', 'Rushing Fumbles Lost', 'Rushing First Downs']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs
0,RB,17,332,"1,811",5.5,83,14,18,106.5,3,2,107
1,RB,14,228,"1,259",5.5,70,12,8,89.9,2,1,61
2,RB,16,292,"1,205",4.1,32,6,13,75.3,1,1,60
3,RB,17,307,"1,200",3.9,37,6,7,70.6,0,0,62
4,RB,13,249,"1,159",4.7,66,9,6,89.2,3,2,57
...,...,...,...,...,...,...,...,...,...,...,...,...
301,QB,5,6,-6,-1.0,-1,0,0,-1.2,0,0,0
302,QB,6,9,-7,-0.8,0,0,0,-1.2,0,0,0
303,QB,5,11,-8,-0.7,2,0,0,-1.6,0,0,1
304,QB,5,9,-8,-0.9,2,0,0,-1.6,0,0,0


In [5]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats['Year'] = 2021
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Jonathan Taylor,IND,RB,17,332,"1,811",5.5,83,14,18,106.5,3,2,107,2021
1,2,Nick Chubb,CLE,RB,14,228,"1,259",5.5,70,12,8,89.9,2,1,61,2021
2,3,Joe Mixon,CIN,RB,16,292,"1,205",4.1,32,6,13,75.3,1,1,60,2021
3,4,Najee Harris,PIT,RB,17,307,"1,200",3.9,37,6,7,70.6,0,0,62,2021
4,5,Dalvin Cook,MIN,RB,13,249,"1,159",4.7,66,9,6,89.2,3,2,57,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,302,Logan Woodside,TEN,QB,5,6,-6,-1.0,-1,0,0,-1.2,0,0,0,2021
302,303,Blaine Gabbert,TB,QB,6,9,-7,-0.8,0,0,0,-1.2,0,0,0,2021
303,304,Brian Hoyer,NE,QB,5,11,-8,-0.7,2,0,0,-1.6,0,0,1,2021
304,304,Cooper Rush,DAL,QB,5,9,-8,-0.9,2,0,0,-1.6,0,0,0,2021


In [6]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Jonathan Taylor,IND,RB,17,332,"1,811",5.5,83,14,18,106.5,3,2,107,2021
1,2,Nick Chubb,CLE,RB,14,228,"1,259",5.5,70,12,8,89.9,2,1,61,2021
2,3,Joe Mixon,CIN,RB,16,292,"1,205",4.1,32,6,13,75.3,1,1,60,2021
3,4,Najee Harris,PIT,RB,17,307,"1,200",3.9,37,6,7,70.6,0,0,62,2021
4,5,Dalvin Cook,MIN,RB,13,249,"1,159",4.7,66,9,6,89.2,3,2,57,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,302,Logan Woodside,TEN,QB,5,6,-6,-1.0,-1,0,0,-1.2,0,0,0,2021
302,303,Blaine Gabbert,TB,QB,6,9,-7,-0.8,0,0,0,-1.2,0,0,0,2021
303,304,Brian Hoyer,NE,QB,5,11,-8,-0.7,2,0,0,-1.6,0,0,1,2021
304,304,Cooper Rush,DAL,QB,5,9,-8,-0.9,2,0,0,-1.6,0,0,0,2021


In [7]:
total_stats.to_csv("Rushing_2021.csv", index = False)
print("Data saved to Rushing_2021.csv")

Data saved to Rushing_2021.csv
